# Epsilon

In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', optimized=True, numThreads=4, quiet=True)
import simuPOP as sim
import pandas as pd
from saegus import breed, operators, simulate, analyze, parse, parameters
import shelve
import numpy as np
import random
import h5py
import collections as col
np.set_printoptions(suppress=True, precision=3)

In [2]:
mg = analyze.MultiGeneration('epsilon')

In [3]:
run_id = 'epsilon'
generations_of_random_mating = 10
sample_sizes=[250, 500, 750, 1000]
number_of_qtl = 10
number_of_replicates = 6
founders = [[2, 26], [3, 25], [4, 24], [5, 23]]
os_per_pair = 500
recombination_rates = [0.01]*1478

In [4]:
prefounders = sim.loadPopulation('bia_prefounders.pop')

In [5]:
sim.tagID(prefounders, reset=True)

In [6]:
alleles = np.array(pd.read_hdf('parameters/alleles_at_1478_loci.hdf'))

In [7]:
rdm_populations = sim.Simulator(prefounders, 2, stealPops=False)
sel_populations = sim.Simulator(prefounders, 2, stealPops=False)
dri_populations = sim.Simulator(prefounders, 2, stealPops=False)

rdm_magic = breed.MAGIC(rdm_populations, founders, recombination_rates)
sel_magic = breed.MAGIC(sel_populations, founders, recombination_rates)
dri_magic = breed.MAGIC(dri_populations, founders, recombination_rates)

sim.tagID(prefounders, reset=27)

rdm_magic.generate_f_one(founders, os_per_pair)
sel_magic.generate_f_one(founders, os_per_pair)
dri_magic.generate_f_one(founders, os_per_pair)

sim.stat(rdm_populations.population(0), alleleFreq=sim.ALL_AVAIL)
af = analyze.allele_data(rdm_populations.population(0), alleles, list(range(1478)))

In [8]:
minor_alleles = np.asarray(af.minor_allele, dtype=np.int8)

In [9]:
rdm_magic.recombinatorial_convergence(rdm_populations, 4, 500)
sel_magic.recombinatorial_convergence(sel_populations, 4, 500)
dri_magic.recombinatorial_convergence(dri_populations, 4, 500)

Start of recombinatorial convergence.
Prior to convergence: 4
Prior to convergence: 2
Start of recombinatorial convergence.
Prior to convergence: 4
Prior to convergence: 2
Start of recombinatorial convergence.
Prior to convergence: 4
Prior to convergence: 2


In [10]:
study = analyze.Study(run_id)

In [11]:
qtl = sorted(random.sample(tuple(range(1478)), 50))

In [12]:
qtl

[14,
 54,
 77,
 86,
 93,
 94,
 130,
 132,
 153,
 210,
 211,
 242,
 267,
 290,
 338,
 341,
 350,
 353,
 358,
 373,
 377,
 379,
 415,
 418,
 513,
 519,
 572,
 612,
 715,
 721,
 734,
 740,
 804,
 831,
 892,
 924,
 969,
 993,
 1078,
 1099,
 1113,
 1119,
 1128,
 1171,
 1249,
 1314,
 1339,
 1354,
 1371,
 1401]

In [13]:
additive_trait = parameters.Trait()
allele_effects = additive_trait.assign_allele_effects(alleles, qtl, random.expovariate, 1,
                                                     multiplicity=1)

In [14]:
ae_array = additive_trait.convert_allele_effects_into_array(prefounders.totNumLoci(), 6, allele_effects)

In [15]:
ae_array[qtl, :]

array([[ 1.149,  0.   ,  0.   ,  0.338,  0.   ,  0.   ],
       [ 0.964,  0.527,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.85 ,  0.   ,  0.235,  0.   ,  0.   ],
       [ 0.   ,  1.616,  0.   ,  0.404,  0.   ,  0.   ],
       [ 0.065,  0.   ,  0.   ,  3.433,  0.   ,  0.   ],
       [ 0.834,  0.   ,  0.   ,  0.706,  0.   ,  0.   ],
       [ 0.713,  0.   ,  0.232,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.092,  0.   ,  1.224,  0.   ,  0.   ],
       [ 0.   ,  2.743,  0.   ,  2.532,  0.   ,  0.   ],
       [ 0.506,  0.   ,  0.721,  0.   ,  0.   ,  0.   ],
       [ 1.084,  0.   ,  0.205,  0.   ,  0.   ,  0.   ],
       [ 1.552,  0.   ,  1.993,  0.   ,  0.   ,  0.   ],
       [ 6.672,  0.   ,  0.52 ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.178,  0.64 ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.582,  2.184],
       [ 0.   ,  1.119,  0.   ,  1.566,  0.   ,  0.   ],
       [ 0.   ,  0.237,  0.   ,  0.158,  0.   ,  0.   ],
       [ 0.533,  0.029,  0.   ,

In [16]:
ae_array.shape

(1478, 6)

In [17]:
sampling_generations = [i for i in range(2, 10, 2)]
sample_sizes = {i: 100 for i in range(11)}

In [18]:
rdm_meta_populations = {rep: [] for rep in range(2)}
sel_meta_populations = {rep: [] for rep in range(2)}
dri_meta_populations = {rep: [] for rep in range(2)}

In [ ]:
import importlib as imp
imp.reload(simulate)

In [20]:
rdm_mating = simulate.RandomMating(10, 2000, 0.05, 0.5, 5, 0.7, sample_sizes)

In [21]:
drift = simulate.Drift(10, 2000, 0.05, 0.5, 5, 0.7, sample_sizes)

In [22]:
selection = simulate.Truncation(10, 2000, 0.05, 0.5, 5, 0.7, sample_sizes)

In [23]:
rdm_mating.replicate_random_mating(rdm_populations, rdm_meta_populations, qtl, ae_array, recombination_rates)

Initial: Sampled 100 individuals from generation 0 Replicate: 0.
Initial: Sampled 100 individuals from generation 0 Replicate: 1.
Generation: 0
Generation: 0
Generation: 1
Generation: 1
Generation: 2
Generation: 2
Generation: 3
Generation: 3
Generation: 4
Generation: 4
Generation: 5
Generation: 5
Generation: 6
Generation: 6
Generation: 7
Generation: 7
Generation: 8
Generation: 8
Generation: 9
Generation: 9
Final: Sampled 100 individuals from generation 10
Final: Sampled 100 individuals from generation 10


del rdm_populations

In [ ]:
drift.replicate_recurrent_drift(dri_populations, dri_meta_populations, qtl, ae_array, recombination_rates)

In [ ]:
selection.replicate_selection(sel_populations, sel_meta_populations, qtl, ae_array, recombination_rates)

del sel_populations

In [ ]:
wombo_combo = [rdm_meta_populations, dri_meta_populations, sel_meta_populations]

In [24]:
for rep, sample_list in rdm_meta_populations.items():
    for sample in sample_list:
        sim.stat(sample, alleleFreq=sim.ALL_AVAIL)

In [25]:
mg = analyze.MultiGeneration('demeter')

In [ ]:
mg.store_allele_frequency_data(meta_populations, minor_alleles, 'demeter_allele_frequencies.hdf5')

In [ ]:
study.store_allele_frequencies(meta_populations, alleles)

In [ ]:
sets_of_segregating_loci = study.seg_loci_among_samples(meta_populations)

In [ ]:
concordant_segregating_loci = list(sets_of_segregating_loci.keys())[0]

In [ ]:
concordant_segregating_loci

In [ ]:
minor_alleles_at_loci = np.asarray(af.minor_allele)[list(concordant_segregating_loci)]

In [ ]:
concordant_segregating_loci

In [ ]:
minor_alleles.shape

In [ ]:
add_trait = parameters.Trait()

In [ ]:
seg_loc_storage = shelve.open('segregating_loci_storage')
seg_loc_storage[run_id] = concordant_segregating_loci
seg_loc_storage.close()

In [ ]:
droppable_loci = [droppable for droppable in range(1478) if droppable not in concordant_segregating_loci]

In [ ]:
droppable_loci

In [ ]:
import importlib as imp
imp.reload(analyze)

In [ ]:
mg = analyze.MultiGeneration('bia')

In [ ]:
import cProfile

In [ ]:
mg.multiple_sample_analyzer(meta_populations, qtl, allele_effects, minor_alleles, concordant_segregating_loci)

In [ ]:
example_pop = meta_populations[0][0]

In [ ]:
example_ind = example_pop.individual(0)

In [ ]:
exgeno = np.asarray([example_ind.genotype(ploidy=0)[locus] for locus in range(1478)])

In [ ]:
exgeno.shape

In [ ]:
exgeno

In [ ]:
int_to_

In [ ]:
genolookups = np.asarray([['A', 'C', 'G', 'T', 'D', 'I'] for i in range(1478)])

In [ ]:
%timeit genolookups = np.asarray([['A', 'C', 'G', 'T', 'D', 'I'] for i in range(1478)])

In [ ]:
print(genolookups)

In [ ]:
exgeno_alpha = np.asarray(example_ind.genotype(ploidy=0))
exgeno_beta = np.asarray(example_ind.genotype(ploidy=1))

In [ ]:
exgeno_alpha

In [ ]:
exgeno_alpha + exgeno_beta

In [ ]:
argenos_alpha = genolookups[range(1478), exgeno_alpha]
argenos_beta = genolookups[range(1478), exgeno_beta]

In [ ]:
from numpy.core import defchararray

In [ ]:
genotype = defchararray.add(genolookups[range(1478), exgeno_alpha], genolookups[range(1478), exgeno_beta])

In [ ]:
genotype

In [ ]:
sorted_genotype = np.empty(1478, dtype=genotype.dtype)
unsorted_genotype = np.empty(1478, dtype=genotype.dtype)

### Iteration Assignment

In [ ]:
for i in range(1478):
    sorted_genotype[i] = ''.join(sorted(genotype[i]))
    unsorted_genotype[i] = genotype[i]

### Ellipsis Simultaneous Assignment

In [ ]:
sorted_genotype[...] = ''.join(sorted(genotype[...]))

In [ ]:
sorted_genotype

In [ ]:
genotype

In [ ]:
mismatched_loci = np.where(sorted_genotype != unsorted_genotype)

In [ ]:
mismatched_loci

In [ ]:
genotype[mismatched_loci]

In [ ]:
sorted_genotype[mismatched_loci]

In [ ]:
sorted_genotype

In [ ]:
catted_genos = defchararray.add(argenos_alpha, argenos_beta)

In [ ]:
catted = defchararray.add(argenos_alpha, argenos_beta)

In [ ]:
catted

In [ ]:
for i in range(1478):
    nucatted[i] = ''.join(sorted(catted[i]))

In [ ]:
nucatted

In [ ]:
catted.dtype

In [ ]:
nucatted = np.empty(1478, dtype=catted.dtype)

In [ ]:
nucatted.dtype

In [ ]:
example = np.asarray(list(zip(argenos_alpha, argenos_beta)))

In [ ]:
%timeit np.asarray(list(argenos_alpha[i] + argenos_beta[i] for i in range(1478)))

In [ ]:
np.add?

In [ ]:
from numpy.core import defchararray

In [ ]:
thing =  defchararray.add(argenos_alpha, argenos_beta)

In [ ]:
list(map(sorted, thing[:100]))

In [ ]:
genorays 

In [ ]:
map(sorted, catted_strings)

In [ ]:
catted_strings = defchararray.add(exgeno_alpha, exgeno_beta)

In [ ]:
int_to_snp_conversions = {0: 'A', 1: 'C', 2: 'G', 3: 'T', 4: 'D', 5: 'I'}

In [ ]:
alpha_genotype = np.asarray(example_ind.genotype(ploidy=0))
beta_genotype = np.asarray(example_ind.genotype(ploidy=1))

In [ ]:
%timeit [''.join(sorted(int_to_snp_conversions[a]+ int_to_snp_conversions[b])) for a, b in zip(alpha_genotype, beta_genotype)]

In [ ]:
%timeit np.asarray(example_ind.genotype(ploidy=0))

In [ ]:
analyze.store_allele_effect_frequency_tables(meta_population, alleles, 
                                             qtl,
                                             exponential_allele_effects,
                                            run_id, 'exponential')

In [ ]:
loci_conversions = shelve.open(run_id+'_loci_conversions')
saegus_to_tassel_loci = {}
tassel_to_saegus_loci = {}
for idx, locus in enumerate(concordant_segregating_loci):
    saegus_to_tassel_loci[locus] = idx
    tassel_to_saegus_loci[idx] = locus
loci_conversions['saegus_to_tassel'] = saegus_to_tassel_loci
loci_conversions['tassel_to_saegus'] = tassel_to_saegus_loci
loci_conversions.close()

In [ ]:
seg_loc_storage = shelve.open('segregating_loci_storage')
seg_loc_storage['bacchus'] = concordant_segregating_loci
seg_loc_storage.close()

In [ ]:
int_to_snp = {0: 'A', 1: 'C', 2: 'G', 3: 'T', 4: '-', 5: '+'}
snp_to_int = {'A': 0, 'C': 1, '-': 4, 'G': 2, '+': 5, 'T': 3}
conv = shelve.open('synthesis_parameters')
conv['integer_to_snp'] = int_to_snp
conv['snp_to_integer'] = snp_to_int
conv.close()

In [ ]:
exponential_allele_effects_table = analyze.generate_allele_effects_table(qtl, alleles, 
                                                exponential_allele_effects, saegus_to_tassel_loci)

In [ ]:
exponential_allele_effects_table

In [ ]:
import importlib as imp
imp.reload(analyze)

In [ ]:
analyze.remap_allele_frequency_table_loci(analyze.reload_allele_frequencies_table(run_id, 0, 250, 'exponential'), concordant_segregating_loci)

In [ ]:
analyze.write_multiple_sample_analyzer(sample_library, sample_sizes, qtl, alleles, 
                                       exponential_allele_effects, 0.7,  concordant_segregating_loci, 
                                       run_id=run_id, sub_run_id='_exponential', 
                                       allele_frequency_hdf=run_id+'_allele_frequency_storage.h5')

In [ ]:
example_pop = meta_populations[0][0]

In [ ]:
int(max(example_pop.indInfo('generation')))

In [ ]:
def multiple_sample_analyzer(meta_population_library,
                            qtl,
                             allele_effects,
                            minor_alleles,
                             loci):
    for rep_id, sample_list in meta_population_library.items():
        for sample in sample_list:
            gen_id = int(max(sample.indInfo('generation')))
            name = '_'.join(['bia', str(rep_id), str(gen_id)])
            print(name)

In [ ]:
minor_alleles

In [ ]:
multiple_sample_analyzer(meta_populations, qtl, 
                         allele_effects,
                         minor_alleles,
                         concordant_segregating_loci)

In [ ]:
af

In [ ]:
input_dir = os.path.join('C', 'tassel', 'input')

In [ ]:
structure_file_name = '_'.join(['bia', '0', '0', 'structure_matrix.txt'])

In [ ]:
os.path.join('C', 'tassel', 'input', '_'.join(['bia', '0', '0', 'simulated_hapmap.txt']))

In [ ]:
kinship_file_name = '_'.join(['bia', '0', '0', 'kinship_matrix.txt'])

In [ ]:
os.path.join('C:\\tassel\\input', '_'.join(['bia', '0', '0', 'simulated_hapmap.txt']))

In [ ]:
file_out_path = os.path.join('C', 'tassel', 'input', file_out_name)

In [ ]:
file_out_path

In [ ]:
def multiple_sample_analyzer(meta_population_library,
                            qtl,
                             allele_effects,
                            minor_alleles,
                             loci):
    int_to_snp_map = {0: 'A', 1: 'C', 2: 'G', 3: 'T', 4: 'D', 5: 'I'}
    indir = "C:\\tassel\\input"
    
    minor_allele_frequency_file = h5py.File('bia_allele_frequencies.hdf5')
    
    for rep_id, sample_list in meta_population_library.items():
        for sample in sample_list:
            gen_id_name = str(int(max(sample.indInfo('generation'))))
            rep_id_name = str(rep_id)
            
            name = 'bia'+'_'+rep_id_name+'_'+gen_id_name
            
            file_out_name = '_'.join(['bia', rep_id_name, gen_id_name])
            
            af_access_name = '/'.join(['af', rep_id_name, gen_id_name])
            minor_allele_frequencies = minor_allele_frequency_file[af_access_name][list(loci)]
            gwas = analyze.GWAS(sample, concordant_segregating_loci, 'bia')
            
            strut_matrix_file_name = os.path.join(input_dir, '_'.join(['bia', rep_id_name, gen_id_name, 'structure_matrix.txt']))
            hapmap_file_name = os.path.join(input_dir, '_'.join(['bia', rep_id_name, gen_id_name, 'simulated_hapmap.txt']))
            trait_file_name = os.path.join(input_dir, '_'.join(['bia', rep_id_name, gen_id_name, 'phenotype_vector.txt']))
            kinship_file_name = os.path.join(input_dir, '_'.join(['bia', rep_id_name, gen_id_name, 'kinship_matrix.txt']))
            
            ccm = gwas.calculate_count_matrix(minor_alleles)
            ps_svd = gwas.pop_struct_svd(ccm)
            gwas.hapmap_formatter(int_to_snp_map, input_dir)
            gwas.trait_formatter(trait_file_name)
            gwas.calc_kinship_matrix(ccm, minor_allele_frequencies, kinship_file_name)
            gwas.population_structure_formatter(ps_svd, struct_matrix_file_name)
            
            gwas.replicate_tassel_gwas_configs(rep_id_name, gen_id_name,
                                              hapmap_file_name,
                                              kinship_file_name,
                                              trait_file_name,
                                              struct_matrix_file_name,
                                              'C:\\tassel\\output\\bia_'+rep_id_name+'_'+gen_id_name+'_out_',
                                            "C:\\Users\DoubleDanks\\BISB\\wisser\\code\\rjwlab-scripts\\"
                                              "saegus_project\\devel\\magic\\1478\\gwas_pipeline.xml")
    
    minor_allele_frequency_file.close()

In [ ]:
multiple_sample_analyzer(meta_populations, qtl, allele_effects, minor_alleles, concordant_segregating_loci)

In [ ]:
import h5py

In [ ]:
with h5py.File('bia_allele_frequencies.hdf5') as biaf:
    reloaded_af = np.array(biaf[afname])

In [ ]:
fisegloc = list(concordant_segregating_loci)

In [ ]:
minor_allele_frequencies = reloaded_af[fisegloc]

In [ ]:
def write_super_tables(power_and_fpr_raw_data, sample_sizes, number_of_replicates, run_id, sub_run_id=''):
    for size in sample_sizes:
        for rep in range(number_of_replicates):
            name = run_id + '_' + sub_run_id + '_' + str(rep) + '_' + str(size) + '_super_table.txt'
            power_and_fpr_raw_data[size][rep].to_csv(name, sep='\t')

In [ ]:
cd C:\tassel\output

In [ ]:
expo_power_fpr_raw_data = analyze.collect_power_analysis_data(run_id, sample_sizes, number_of_replicates, concordant_segregating_loci, 'exponential')

In [ ]:
expo_power_fpr_raw_data[250]

In [ ]:
write_super_tables(expo_power_fpr_raw_data,
                  sample_sizes,
                  number_of_replicates,
                  'bacchus',
                  sub_run_id='exponential')

In [ ]:
expo_results, expo_true_positives, expo_false_positives = study.calculate_power_fpr(expo_power_fpr_raw_data, sample_sizes, 
                                                                             number_of_replicates, number_of_qtl)

In [ ]:
expo_results

In [ ]:
mean_and_stdev = pd.DataFrame([expo_results.mean(), expo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('bacchus_exponential_mean_and_stdev_power_fpr.csv', sep='\t')

In [ ]:
geo_results

In [ ]:
geometric_allele_effects_table

In [ ]:
exponential_allele_effects_table

In [ ]:
expo_results.to_csv("bacchus_exponential_power_fpr_results.txt", sep='\t')

In [ ]:
mean_and_stdev = pd.DataFrame([geo_results.mean(), geo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('full_icecrown_geometric_mean_and_stdev_power_fpr.txt', sep='\t')

In [ ]:
expo_results, expo_true_positives, expo_false_positives = full_icecrown.calculate_power_fpr(expo_power_fpr_raw_data,
                                                                                      sample_sizes,
                                                                                      number_of_replicates,
                                                                                      number_of_qtl)

In [ ]:
expo_results

In [ ]:
expo_results.to_csv('full_icecrown_exponential_power_fpr_results.txt', sep='\t')

In [ ]:
mean_and_stdev = pd.DataFrame([expo_results.mean(), expo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('full_icecrown_exponential_mean_and_stdev_power_fpr.txt', sep='\t')

In [ ]:
write_super_tables(expo_power_fpr_raw_data, sample_sizes, number_of_replicates, run_id, 'exponential')

In [ ]:
geo_aggregate_estimated_actual = pd.DataFrame([np.array(geo_agg_estimated), np.array(geo_agg_actual)], index=['estimated', 'actual']).T

In [ ]:
geo_aggregate_estimated_actual['estimated'] = geo_aggregate_estimated_actual['estimated'].apply(np.fabs)

In [ ]:
geo_aggregate_estimated_actual

In [ ]:
geo_corr = geo_aggregate_estimated_actual['estimated'].corr(geo_aggregate_estimated_actual['actual'])

In [ ]:
geo_agg_estimated

In [ ]:
aggregate_estimated_actual

In [ ]:
geo_corr

In [ ]:
pwd

In [ ]:
geo_aggregate_estimated_actual.to_csv('full_icecrown_geometric_estimated_vs_actual_allele_effects.txt', sep='\t')

In [ ]:
agg_estimated = []
agg_actual = []

In [ ]:
for rep in reps:
    for size in sample_sizes:
        sutable = sutable_collection[rep][size]
        droppable = list(sutable.ix[sutable.ix[:, 'difference'] == 0.0].index)
        qtloci = sutable.drop(droppable, axis=0)
        agg_estimated.extend(list(qtloci['add_effect']))
        agg_actual.extend(list(qtloci['difference']))

In [ ]:
aggregate_estimated_actual = pd.DataFrame([np.array(agg_estimated), np.array(agg_actual)], index=['estimated', 'actual']).T

In [ ]:
aggregate_estimated_actual['estimated'] = np.fabs(aggregate_estimated_actual['estimated'])

In [ ]:
aggregate_estimated_actual

In [ ]:
correlation_actual_vs_effects = aggregate_estimated_actual['estimated'].corr(aggregate_estimated_actual['actual'])

In [ ]:
aggregate_estimated_actual.to_csv('full_icecrown_exponential_estimated_vs_actual_allele_effects.txt', sep='\t')

In [ ]:
aggregate_estimated_actual['estimated'] = np.fabs(aggregate_estimated_actual['estimated'])

In [ ]:
cd C:\tassel\output\full_icecrown\exponential

In [ ]:
expo_estimated_actual = pd.read_csv('full_icecrown_exponential_estimated_vs_actual_allele_effects.txt', sep='\t', index_col=0)

In [ ]:
expo_estimated_actual

In [ ]:
aggregate_estimated_actual

In [ ]:
, from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

In [ ]:
output_notebook()

In [ ]:
aggregate_estimated_actual

In [ ]:
geo_x = aggregate_estimated_actual['estimated']
geo_y = aggregate_estimated_actual['actual']

In [ ]:
p = figure(title="Estimated vs Actual Allele Effects - Geometric Series", 
           title_text_font_size="16",
          x_range=(-0.2, 4))

In [ ]:
p.scatter(geo_x, y, x="Estimated", y="Actual")

p.xaxis.axis_label = "Estimated"
p.yaxis.axis_label = "Actual"

In [ ]:
show(p)

In [ ]:
expo

In [ ]:
p = figure(title="Estimated vs Actual Allele Effects - Geometric Series", title_text_font_size="16")

In [ ]:
expo_plot = figure(title="Estimated vs Actual Effects - Exponential(lambda=1)", 
                   title_text_font_size="16", 
                  x_range=(0, 4))

x = np.array(expo_estimated_actual['estimated'])
y = np.array(expo_estimated_actual['actual'])

expo_plot.xaxis.axis_label = "Estimated"
expo_plot.yaxis.axis_label = "Actual"

In [ ]:
expo_plot.scatter(x, y)

In [ ]:
show(expo_plot)

In [ ]:
from bokeh.io import hplot

In [ ]:
geo_plot = figure(title="Estimated vs Actual Allele Effects - Geometric Series", 
           title_text_font_size="16",
          x_range=(0, 4), y_range=(0, 4))

In [ ]:
geo_x = aggregate_estimated_actual['actual']
geo_y = aggregate_estimated_actual['estimated']

In [ ]:
geo_plot.xaxis.axis_label = "Actual"
geo_plot.yaxis.axis_label = "Estimated"
geo_plot.scatter(geo_x, geo_y, x="Actual", y="Estimated")

In [ ]:
expo_plot = figure(title="Estimated vs Actual Effects - Exponential(lambda=1)", 
                   title_text_font_size="16", 
                  x_range=(0, 4), y_range=(0, 4))

expo_x = np.array(expo_estimated_actual['actual'])
expo_y = np.array(expo_estimated_actual['estimated'])

expo_plot.xaxis.axis_label = "Actual"
expo_plot.yaxis.axis_label = "Estimated"
expo_plot.scatter(expo_x, expo_y)

In [ ]:
multi_plot = hplot(geo_plot, expo_plot)
show(multi_plot)

In [ ]:
output_file("multi_plot.png")

In [ ]:
ls